In [1]:
import torch
from LegibilityModel import LegibilityModel
from transformers import TrOCRProcessor, AutoModel
from datasets import load_dataset
from utils import Renderer
from Trainer import MultiTaskTrainer
from Metrics import binary_classification_metric
import numpy as np

In [2]:
preprocessor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
model = LegibilityModel(config=None)
weights = torch.load('pytorch_model.bin', map_location=torch.device('cpu'))
model.load_state_dict(weights)
model.register_for_auto_class()
model.save_pretrained('./saved')

In [ ]:
model = AutoModel.from_pretrained('./saved', trust_remote_code=True)

In [3]:
dataset = load_dataset('dvsth/LEGIT')

Using custom data configuration dvsth--LEGIT-d84a4d72774d3652
Found cached dataset parquet (/Users/dvsth/.cache/huggingface/datasets/dvsth___parquet/dvsth--LEGIT-d84a4d72774d3652/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def preprocess0(example):
    return {'i0': preprocessor(example['img0'], return_tensors='pt').pixel_values}

def preprocess1(example):
    return {'i1': preprocessor(example['img1'], return_tensors='pt').pixel_values}

In [5]:
dataset = dataset['test'].map(preprocess0, batched=True, batch_size=2000)

Parameter 'function'=<function preprocess0 at 0x14f5e9af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
dataset = dataset.map(preprocess1, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
trainer = MultiTaskTrainer(
    model=model,
    compute_metrics=binary_classification_metric
)

In [ ]:
trainer.predict(dataset['test'])